# Recurrent neural networks, 1D CNN

RNN's are well suited for processing data which consists of sequences. These models process sequence element by element, while keeping the memory of the elements that the network has already seen. 
One dimensional convolutional layers can also be applied to sequences, same as they are applied to 2d images. 

### Dataset used

IMDB sentiment recognition dataset. 

In [2]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

from keras.models import Model
from keras.layers import Dense, GRU, Input, Conv1D, MaxPool1D, Embedding, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.datasets import imdb
from keras.preprocessing import sequence

# prepare the data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=20000)
X_train = sequence.pad_sequences(X_train, maxlen=80)
X_test = sequence.pad_sequences(X_test, maxlen=80)

In [3]:
print(X_train.shape)

(25000, 80)


In [ ]:
x = Input(shape=X_train[0].shape)
h = x
h = Embedding(20000, 128)(h)

mode = 'gru'

if mode == 'gru':
    h = GRU(64)(h)
else:
    h = Conv1D(32, 5, strides=5)(h)
    h = MaxPool1D(5)(h)
    h = LeakyReLU()(h)
    
    h = Flatten()(h)
    
h = Dense(2, activation='softmax')(h)

# get the trivial accuracy
dummy_model = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
dummy_score = dummy_model.score(X_test, y_test)
print('Dummy test accuracy:', dummy_score)

model = Model(inputs=x, outputs=h)

# compile computational graph of NN
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# train NN
model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=1,
    verbose=1
)

# evaluate your NN
score = model.evaluate(X_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])